# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
19/08/2024

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import cv2
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# des beaux graphiques
import seaborn as sns

In [41]:
%matplotlib
%matplotlib

Using matplotlib backend: TkAgg
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [42]:
# SENSIBILITE CAPTEUR
# S_force_bs =       # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

# Fonction de traitement

In [43]:
# ENTREES :
# position_y : tableau des positions de y
# position_z : tableau des positions de z
# force :  tableau des forces
# position_y, position_z et force sont issus des datas au fromat .txt aqui par python sous la forme "#y  z  offset moyenne_brute(N) moyenne(N) "
# SORTIE 
# pentes = tableau de pentes (raideur)
# deb = point de départ de la section d'ajustement linéaire

def calcul_pentes(position_y, position_z, force): 
    pentes = [] # initialisation du vecteur contenant les pentes
    deb = np.where(force > 0.2*max(force))[0][0] # indice du début de la zone d'intérêt : pour être certain.e que l'on est dans la partie linéaire, qu'on considère comme 20% de la valeur max
    
    for m in (np.unique(position_y, return_counts=True)[0]) :  # boucle selectionnant chaque valeur différente de position y
        y  = position_y == m 
        z = position_z[y]
        F = force[y]
        
        coefficients = np.polyfit(z[deb:], F[deb:], 1)   # curve fitting par ajustement polynomial d'ordre 1
        pente = round(coefficients[0],2) # extraction de la pente arrondi à 2 chiffres significatifs
        pentes.append(pente) # une pente est défini et rangée dans le vecteur "pentes" pour chaque postion y différente

    return pentes, deb

In [44]:
# ENTREE
# position_y : tableau des positions de y
# position_z : tableau des positions de z
# force :  tableau des forces
# SORTIE
# Vecteur des 3 coefficients paramétrant l'approxymation d'ordre 2
from sklearn.metrics import r2_score
def modelisation_profile_de_raideur(position_y, position_z, Force) :
    pos_y = np.unique(position_y)
    pentes, deb = calcul_pentes(position_y, position_z, Force)
    
    coef = np.polyfit(np.unique(pos_y), pentes, 2)
    poly = PolynomialFeatures(degree=2)
    poly_y = poly.fit_transform(np.array(pos_y).reshape(-1,1))
    pentes = np.array(pentes)
    reg_model = LinearRegression().fit(poly_y, pentes)
    coef = np.polyfit(pos_y, pentes, 2)
    R2 = reg_model.score(poly_y, pentes)
    
    # print(coef, R2)
    
    return coef, R2


In [45]:
# ENTREE
# position_y : tableau des positions de y
# position_z : tableau des positions de z
# force :  tableau des forces
# SORTIE
# R2



def plot_profile(position_y, position_z, Force, nom_anche) :
    pos_y = np.unique(position_y)
    pentes, deb = calcul_pentes(position_y, position_z, Force)
    
    coef = np.polyfit(np.unique(pos_y), pentes, 2)
    poly = PolynomialFeatures(degree=2)
    poly_y = poly.fit_transform(np.array(pos_y).reshape(-1,1))
    pentes = np.array(pentes)
    reg_model = LinearRegression().fit(poly_y, pentes)
    coef = np.polyfit(pos_y, pentes, 2)
    R2 = reg_model.score(poly_y, pentes)

    mymap = plt.get_cmap("Blues")
    profile_only = False
    if profile_only :
        plt.figure()
        plt.title(f'Profile de raideur \nnombre de points : {len(pos_y)*len(np.unique(position_z))}')
        plt.plot(pos_y, pentes, '.', color=mymap(0.25), label='données expérimentales')
        plt.plot(pos_y, np.polyval(coef, pos_y), linestyle='-', color='orange', label=f'y = {coef[0]:.2f}x² + {coef[1]:.2f}x + {coef[2]:.2f} \nR² = {round(R2,3)}')
        plt.xlabel('Coordonnée y (mm)', fontsize=16)
        plt.ylabel('Raideur $k=F/z$ (N/mm)', fontsize=16)
        plt.legend()
        plt.grid()
        
    profile_et_pentes = True
    if profile_et_pentes : 
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))

        for idx, m in enumerate(np.unique(position_y, return_counts=True)[0]):
            couleur = mymap((idx+4) / len(np.unique(position_y)))
            y = position_y == m
            z = position_z[y]
            F = Force[y]
            coefficients = np.polyfit(z[deb:], F[deb:], 1)
            axs[0].plot(z, F, '.', label=f'y = {m} mm', color=couleur)
            axs[0].plot(z[deb:], np.polyval(coefficients, z[deb:]), linestyle='-', color=couleur, label=f'pente = {np.round(coefficients[0], 2)}')
            pente = coefficients[0]
            pente = round(pente, 2)
            axs[1].plot(m, pente, '.', color=couleur, label='')

        axs[0].set_title(f'Force mesurée en fonction de la postion z \npour différents point y')
        axs[0].legend(bbox_to_anchor=(-0.5, 0.5), loc='center left', fontsize=10, borderaxespad=0)
        axs[0].set_xlabel('Coordonnée z (mm)', fontsize=14)
        axs[0].set_ylabel('Force (N)', fontsize=14)
        axs[0].grid()

        # Ajouter un titre global
        fig.suptitle(f'Raideur par Banc Statique Anche N° {nom_anche} \n\nNombre de points : {len(pos_y) * len(np.unique(position_z))}', fontsize=16)

        axs[1].set_title('Profile de raideur')
        axs[1].plot(pos_y, np.polyval(coef, pos_y), linestyle='-', color='orange', label=f'Modèle : y = {coef[0]:.2f}x² + {coef[1]:.2f}x + {coef[2]:.2f} \nR² = {round(R2, 3)}')
        axs[1].set_xlabel('Coordonnée y (mm)', fontsize=14)
        axs[1].set_ylabel('Raideur $k=F/z$ (N/mm)', fontsize=14)
        axs[1].legend()
        axs[1].grid()

        # Utilisation de tight_layout pour ajuster l'espace
        plt.tight_layout(rect=[0, 0, 1, 0.95]) 
        plt.show()
    
    return R2

# R2 = plot_profile(position_y, position_z, forces)
# print(R2)

# datas du 30/05/2024
Une anche de force 3 1/2 mesurée avec 13*21pt, 40min

In [46]:
# LOAD DATAs

# matrice vide
data = []
pentes = []

# labels 
anches = ['Force 3 1/2']
mymap = plt.get_cmap("Spectral")

# load data
data = pd.read_table('2024.05.30/data_anche1.txt', sep=' ', header=1).values # contient : [y, z, offset, moyenne_brute, moyenne(N) en grandeur physique]
position_y = data[:,0]
position_z = data[:,1]
forces = data[:,4]

In [47]:
# plt.close('all')
R2 = plot_profile(position_y, position_z, forces, 'anche 1')
# coef, R2 = modelisation_profile_de_raideur(position_y, position_z, forces)

In [48]:
plt.close('all')

data = pd.read_table('2024.05.30/data_anche1.txt', sep=' ', header=1).values # contient : [y, z, offset, moyenne_brute, moyenne(N) en grandeur physique]
position_y = data[:,0]
position_z = data[:,1]
forces = data[:,4]

# nombre de points sur la largeur de l'anche
ys, _ = np.unique(position_y, return_counts=True)
nbr_y = ys.shape[0]


ys_13 = ys
ys_11 = [ys[i] for i in [0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12]]
ys_9 = [ys[i] for i in [0, 1, 3, 4, 6, 8, 9, 11, 12]]
ys_7 = [ys[i] for i in [0, 2, 4, 6, 8, 10, 12]]
# ys_5 = [ys[i] for i in [0, 3, 6, 9, 12]]


Y = [ys_7, ys_9, ys_11, ys_13]
nbr_Y = ['7', '9', '11', '13']

# nombre de points sur la largeur de l'anche
zs, _ = np.unique(position_z, return_counts=True)
nbr_z = zs.shape[0]


zs_21 = zs
zs_18 = [zs[i] for i in [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 19, 20]]
zs_16 = [zs[i] for i in [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20]]
zs_14 = [zs[i] for i in [0, 1, 3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19]]
zs_11 = [zs[i] for i in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]]
zs_8 = [zs[i] for i in [0, 3, 6, 9, 12, 15, 18, 20]]

Z = [zs_8, zs_11, zs_14, zs_16, zs_18, zs_21]
nbr_Z = ['8', '11', '14', '16', '18', '21']

affichage = False
if affichage:
    print('ys_13 =', ys_13)
    print('ys_11 =', ys_11)
    print('ys_9 =', ys_9)
    print('ys_7 =', ys_7)
    print('ys_5 =', ys_5)
    print('Y =', Y)
    
if affichage:
    print('zs_21 =', zs_21)
    print('zs_18 =', len(zs_18))
    print('zs_16 =', len(zs_16))
    print('zs_14 =',len(zs_14))
    print('zs_11 =',len(zs_11))
    print('zs_7 =',len(zs_8))
    print('Z =', len(Z))     

# A faire : au lieu de faire les R2, plutot faire la différence entre le modèle idéal (le nombre max de pts) et le modèle avec moins de pts (pour chaque itération)

In [50]:
# Plot les profils pour chaque cas :
plot_profiles = False
# Plot une matrice indiquant la précision en fonction du nombres de points des cas étudiés
plot_precision = True


plt.close('all')
anches = ['Force 3 1/2']
temps = 40 # min
nbr_pts_total = len(np.unique(position_y))* len(np.unique(position_z))
temps_point = temps/nbr_pts_total # min
# print(nbr_pts_total)

len_Y = len(np.array(Y, dtype=object))
len_Z = len(np.array(Z, dtype=object))


for A in anches:
    # vecteurs contenant
    R2s=np.zeros((len_Z, len_Y))
    temps_mesures=np.zeros((len_Z, len_Y))
    nbr_pts_Y=np.zeros((len_Z, len_Y))
    nbr_pts_Z=np.zeros((len_Z, len_Y))
    
    for idx_z, z in enumerate(Z) :
        # ecrémage des datas suivant z
        mask_z = np.isin(position_z, z)
        pos_y_mz = np.array(position_y[mask_z])
        pos_z_mz = np.array(position_z[mask_z])
        f_mz = np.array(forces[mask_z])
        
        for idx_y, y in enumerate(Y):
            # ecrémage des datas suivant y
            mask_y = np.isin(pos_y_mz, y)
            pos_y_mzy = np.array(pos_y_mz[mask_y])
            pos_z_mzy = np.array(pos_z_mz[mask_y])
            f_mzy = np.array(f_mz[mask_y])
            
            # calcul du modèle et de sa précision
            if plot_profiles :
                R2 = plot_profile(pos_y_mzy, pos_z_mzy, f_mzy, 'anche 1')
            if plot_precision:
                coef, R2 = modelisation_profile_de_raideur(pos_y_mzy, pos_z_mzy, f_mzy)
            
            # remplir les matrices 
            R2s[(idx_z, idx_y)]=R2
            nbr_pts_y = len(np.unique(pos_y_mzy))
            # print(nbr_pts_y)
            nbr_pts_z =  len(np.unique(pos_z_mzy))
            # print(nbr_pts_z)
            temps_mesure = nbr_pts_y*nbr_pts_z*temps_point
            temps_mesures[(idx_z, idx_y)]= temps_mesure
            nbr_pts_Y[(idx_z, idx_y)]= nbr_pts_y
            nbr_pts_Z[(idx_z, idx_y)]= nbr_pts_z
            
    if plot_precision :
        plt.figure(figsize=(10, 8))
        sns.heatmap(R2s, annot=True, fmt='.4f', cmap='Spectral', cbar=True,
                    xticklabels=[f'{np.array(nbr_Y[::-1][i], dtype=object)}' for i in range(len(np.array(nbr_Y, dtype=object)))],
                    yticklabels=[f'{np.array(nbr_Z[i])}' for i in range(len(np.array(nbr_Z, dtype=object)))])

        plt.title('Valeurs de $R^2$ en fonction des points de mesure selon y et z')
        plt.xlabel('Nombres de points selon Y')
        plt.ylabel('Nombres de points selon Z')       

In [51]:
plt.close('all')
anches = ['Force 3 1/2']
temps = 40 # min
nbr_pts_total = len(np.unique(position_y))* len(np.unique(position_z))
temps_point = temps/nbr_pts_total # min
# print(nbr_pts_total)

len_Y = len(np.array(Y, dtype=object))
len_Z = len(np.array(Z, dtype=object))


for A in anches:
    # vecteurs contenant
    R2s=np.zeros((len_Z, len_Y))
    temps_mesures=np.zeros((len_Z, len_Y))
    nbr_pts_Y=np.zeros((len_Z, len_Y))
    nbr_pts_Z=np.zeros((len_Z, len_Y))
    nbr_pts_YZ=np.zeros((len_Z, len_Y))
    
    for idx_z, z in enumerate(Z) :
        # ecrémage des datas suivant z
        mask_z = np.isin(position_z, z)
        pos_y_mz = np.array(position_y[mask_z])
        pos_z_mz = np.array(position_z[mask_z])
        f_mz = np.array(forces[mask_z])
        
        for idx_y, y in enumerate(Y):
            # ecrémage des datas suivant y
            mask_y = np.isin(pos_y_mz, y)
            pos_y_mzy = np.array(pos_y_mz[mask_y])
            pos_z_mzy = np.array(pos_z_mz[mask_y])
            f_mzy = np.array(f_mz[mask_y])
            
            # calcul du modèle et de sa précision
            coef, R2 = modelisation_profile_de_raideur(pos_y_mzy, pos_z_mzy, f_mzy)
            
            # remplir les matrices 
            R2s[(idx_z, idx_y)]=R2
            nbr_pts_y = len(np.unique(pos_y_mzy))
            # print(nbr_pts_y)
            nbr_pts_z =  len(np.unique(pos_z_mzy))
            # print(nbr_pts_z)
            nbr_pts_yz = nbr_pts_y*nbr_pts_z
            temps_mesure = nbr_pts_yz*temps_point
            temps_mesures[(idx_z, idx_y)]= temps_mesure
            nbr_pts_Y[(idx_z, idx_y)]= nbr_pts_y
            nbr_pts_Z[(idx_z, idx_y)]= nbr_pts_z
            nbr_pts_YZ[(idx_z, idx_y)]= nbr_pts_yz
            
            # print(f'mesure {[R2, nbr_pts_y, nbr_pts_z, temps_mesure]}')
            
            
plot_scatter_temps_nbre_pts = False
if plot_scatter_temps_nbre_pts :
    data = pd.DataFrame({
        'Temps de Mesure (min)': temps_mesures.flatten(),
        'Précision (R²)': R2s.flatten(),
        'Nombre de points suivant y': nbr_pts_Y.flatten(),
        'Nombre de points suivant z': nbr_pts_Z.flatten(), 
        'Nombre de points total': nbr_pts_YZ.flatten()
        })      
            
    sns.scatterplot(x='Temps de Mesure (min)', y='Précision (R²)', data=data, hue='Nombre de points total', palette='viridis', size='Précision (R²)', sizes=(10, 600), legend="brief")
                
    # Ajouter des lignes de tendance
    sns.regplot(x='Temps de Mesure (min)', y='Précision (R²)', data=data, scatter=False, color='red', line_kws={'label': 'Tendance'})

    # Ajouter des titres et des labels
    plt.title('Relation entre Temps de Mesure et Précision (R²)')
    plt.xlabel('Temps de Mesure (min)')
    plt.ylabel('Précision (R²)')

    # Afficher la légende
    plt.legend()
    plt.grid(True)
    
    
    
##############################
    
data = pd.DataFrame({
    'Temps de Mesure (min)': temps_mesures.flatten(),
    'Précision (R²)': R2s.flatten(),
    'Nombre de points suivant y': nbr_pts_Y.flatten(),
    'Nombre de points suivant z': nbr_pts_Z.flatten()
    }) 
# print(data)

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = data['Nombre de points suivant y']
y = data['Nombre de points suivant z']
z = data['Temps de Mesure (min)']
accuracy = data['Précision (R²)']

ax.set_xlabel("Nombre de points suivant y")
ax.set_ylabel("Nombre de points suivant z")
ax.set_zlabel("Temps de Mesure (min)")

scatter = ax.scatter(x, y, z, c=accuracy, cmap='viridis', s=50)

cbar = plt.colorbar(scatter)
cbar.set_label('Précision (R²)')

plt.show()

